# Cannibalization Study

#### Objective of this notebook is to identify the base store for each of our members 

Logic of the base store is determined based on the no. of transaction in the store 

Importing extracted dataset into our notebook 

In [1]:
import os

dir_ = os.getcwd()

In [2]:
import pandas as pd

Data = pd.read_csv(str(dir_)+'/data/cannibalization_dataset_v7.csv', parse_dates=['the_date_transaction'])
#Data = pd.read_csv('cannibalization_dataset_v6.csv', parse_dates=['the_date_transaction'])


C:\Users\AAMIN01\AppData\Local\Temp\ipykernel_19684\1733199692.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  Data = pd.read_csv(str(dir_)+'/data/cannibalization_dataset_v7.csv', parse_dates=['the_date_transaction'])


In [3]:
Data.head()

,ctm_customer_id,the_date_transaction,the_transaction_id,f_qty_item,f_to_tax_in,offline_code_postal,store,website_order_card_number,website_postcode_approx,final_approx_postcode
0,2092544449924,2021-11-13 16:49:35+00:00,7-2000-2000-20211113164935-109-9893,9,341.0,NaN,6. SA,NaN,NaN,NaN
1,2092508537056,2022-07-10 15:03:32+00:00,7-2000-2000-20220710150332-109-5171,4,192.0,NaN,6. SA,NaN,NaN,NaN
2,2092545685031,2021-11-14 15:48:08+00:00,7-2000-2000-20211114154808-202-459,6,206.0,NaN,6. SA,NaN,NaN,NaN
3,2091585656186,2022-08-11 14:21:17+00:00,7-2000-2000-20220811142117-202-5710,6,230.0,NaN,6. SA,NaN,NaN,NaN
4,2092240013818,2021-02-21 17:46:21+00:00,7-2000-2000-20210221174621-107-4480,6,159.0,NaN,6. SA,NaN,NaN,NaN


## Filter out transactions with TO <= 0 and f_qty_item <= 0

For the base store identification , we create a temporary dataframe to work on 

In [191]:
Temp_Data=Data.drop(['offline_code_postal', 'website_order_card_number','website_postcode_approx','final_approx_postcode'], axis=1)

In [192]:
# filter rows with turnover > 0 i.e. sales, not return 
Temp_Data = Temp_Data[(Temp_Data['f_to_tax_in'] > 0) & (Temp_Data['f_qty_item'] > 0)]
print(Temp_Data.describe())
Temp_Data.head()

       f_qty_item  f_to_tax_in
count  5050171.00   5050171.00
mean         4.18       165.85
std          5.57       233.96
min          1.00         0.00
25%          1.00        49.00
50%          3.00        99.00
75%          5.00       199.00
max       3001.00     39530.00


,ctm_customer_id,the_date_transaction,the_transaction_id,f_qty_item,f_to_tax_in,store
0,2092544449924,2021-11-13 16:49:35+00:00,7-2000-2000-20211113164935-109-9893,9,341.00,6. SA
1,2092508537056,2022-07-10 15:03:32+00:00,7-2000-2000-20220710150332-109-5171,4,192.00,6. SA
2,2092545685031,2021-11-14 15:48:08+00:00,7-2000-2000-20211114154808-202-459,6,206.00,6. SA
3,2091585656186,2022-08-11 14:21:17+00:00,7-2000-2000-20220811142117-202-5710,6,230.00,6. SA
4,2092240013818,2021-02-21 17:46:21+00:00,7-2000-2000-20210221174621-107-4480,6,159.00,6. SA


#### Checking on Outliers 

In [193]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

nb_trx_per_customer_Temp_Data = Temp_Data.groupby("ctm_customer_id").agg({"the_transaction_id": pd.Series.nunique})
nb_trx_per_customer_Temp_Data.describe([0.9,0.99, 0.999])

,the_transaction_id
count,2002113.00
mean,2.52
std,28.30
min,1.00
50%,1.00
90%,5.00
99%,16.00
99.9%,35.00
max,35016.00


In [194]:

import numpy as np
nb_trx_per_customer_at_995_percentile = np.percentile(nb_trx_per_customer_Temp_Data['the_transaction_id'],99.5)

In [195]:
print("Remove outlier - top 0.5% of the customers")
nb_trx_per_customer_Temp_Data = nb_trx_per_customer_Temp_Data[~(nb_trx_per_customer_Temp_Data.the_transaction_id > nb_trx_per_customer_at_995_percentile)].reset_index()
print("Cleaned")
print("Nb of purchases per customer within the time period of the data:")
nb_trx_per_customer_Temp_Data.describe([0.6,0.7,0.8,0.9])

Remove outlier - top 0.5% of the customers
Cleaned
Nb of purchases per customer within the time period of the data:


,the_transaction_id
count,1993109.00
mean,2.32
std,2.54
min,1.00
50%,1.00
60%,2.00
70%,2.00
80%,3.00
90%,5.00
max,21.00


In [196]:
nb_trx_per_customer_Temp_Data

,ctm_customer_id,the_transaction_id
0,209396044011.00,1
1,919820970660,1
2,2090008025844,1
3,2090010050803,2
4,2090016565691,4
...,...,...
1993104,2098001116286,1
1993105,2098001911188,7
1993106,2098001913076,1
1993107,2098002664533,1


In [197]:
Temp_Data = Temp_Data[Temp_Data['ctm_customer_id'].isin(nb_trx_per_customer_Temp_Data['ctm_customer_id'])]

In [198]:
Temp_Data.head(10)

,ctm_customer_id,the_date_transaction,the_transaction_id,f_qty_item,f_to_tax_in,store
0,2092544449924,2021-11-13 16:49:35+00:00,7-2000-2000-20211113164935-109-9893,9,341.00,6. SA
1,2092508537056,2022-07-10 15:03:32+00:00,7-2000-2000-20220710150332-109-5171,4,192.00,6. SA
2,2092545685031,2021-11-14 15:48:08+00:00,7-2000-2000-20211114154808-202-459,6,206.00,6. SA
3,2091585656186,2022-08-11 14:21:17+00:00,7-2000-2000-20220811142117-202-5710,6,230.00,6. SA
4,2092240013818,2021-02-21 17:46:21+00:00,7-2000-2000-20210221174621-107-4480,6,159.00,6. SA
5,2091582288588,2022-11-20 19:05:15+00:00,7-2000-2000-20221120190515-205-4969,11,333.00,6. SA
6,2092891001158,2022-09-23 18:00:29+00:00,7-2000-2000-20220923180029-110-3508,5,155.00,6. SA
7,2092721764819,2022-05-01 13:56:52+00:00,7-2000-2000-20220501135652-205-5503,7,215.00,6. SA
9,2091585548566,2022-08-09 21:45:10+00:00,7-2000-2000-20220809214510-203-258,3,94.00,6. SA
10,2092055104855,2022-09-03 20:44:38+00:00,7-2000-2000-20220903204438-206-1537,6,370.00,6. SA


In [199]:
print(Temp_Data.dtypes)

ctm_customer_id                      object
the_date_transaction    datetime64[ns, UTC]
the_transaction_id                   object
f_qty_item                            int64
f_to_tax_in                         float64
store                                object
dtype: object


We would like to adjust the timeframe of the data taking from the beginning of the time till end of 2022

In [200]:
# convert the date column to datetime format
#Temp_Data['the_date_transaction'] = pd.to_datetime(Temp_Data['the_date_transaction'])
#Temp_Data['the_date_transaction'] = pd.to_datetime(Temp_Data['the_date_transaction'], infer_datetime_format=True)


Please adjust the timestamp accordingly based on desired setting~

In [201]:
start_date = '2022-11-01'
end_date = '2022-11-30'

dates = (Temp_Data['the_date_transaction'] > start_date) & (Temp_Data['the_date_transaction'] <= end_date)

Temp_Data = Temp_Data.loc[dates]

Converting the Customer ID to Integer to avoid issues 

In [202]:
def remove_decimal(num_str):
    try:
        return str(int(float(num_str)))
    except:
        return num_str


In [203]:
Temp_Data['ctm_customer_id']=Temp_Data['ctm_customer_id'].apply(remove_decimal)

Understanding our Data - checking on transaction /sales/ members by store

In [204]:
# Group the data by store and count the number of transactions
store_transactions = Temp_Data.groupby('store')['the_transaction_id'].nunique()

# Group the data by store and sum the sales generated
store_sales = Temp_Data.groupby('store')['f_to_tax_in'].sum()

# Group the data by store and count the number of unique members
Members =Temp_Data['ctm_customer_id'].nunique()


In [205]:
store_sales

store
1. Decathlon.my    951437.43
10. MyTown        2498991.00
11. KLCC          1783785.00
12. Puchong       1096065.00
13. Cyberjaya     2214337.00
3. BSD            2964962.00
4. USJ            1381843.00
5. OKR            1016447.00
6. SA             3589440.01
7. PJ             1886107.00
8. KL EAST        1640847.00
st                     78.00
Name: f_to_tax_in, dtype: float64

In [206]:
store_transactions

store
1. Decathlon.my     6057
10. MyTown         22078
11. KLCC           11985
12. Puchong         8669
13. Cyberjaya      17915
3. BSD             16484
4. USJ             10514
5. OKR              7568
6. SA              21070
7. PJ              11421
8. KL EAST         14214
st                     1
Name: the_transaction_id, dtype: int64

In [207]:
Members

126304

In [208]:
# Create a new column to count the number of transactions per customer per store
Temp_Data['transactions_per_store'] = Temp_Data.groupby(['ctm_customer_id', 'store'])['the_transaction_id'].transform('nunique')

In [209]:
# Group the data by store and customer ID, and count the number of transactions
store_customer_transactions = Temp_Data.groupby(['ctm_customer_id','store'])['the_transaction_id'].nunique()
store_customer_transactions.head(100)

ctm_customer_id  store          
2090550311754    1. Decathlon.my    1
2090577786009    3. BSD             1
2090629397726    4. USJ             1
2090629795355    4. USJ             1
2090630810429    7. PJ              1
                                   ..
2090697610796    13. Cyberjaya      1
2090697615289    7. PJ              1
2090697623277    1. Decathlon.my    1
2090697631005    3. BSD             1
2090697846362    10. MyTown         1
Name: the_transaction_id, Length: 100, dtype: int64

In [210]:
print(store_customer_transactions)

ctm_customer_id  store          
2090550311754    1. Decathlon.my    1
2090577786009    3. BSD             1
2090629397726    4. USJ             1
2090629795355    4. USJ             1
2090630810429    7. PJ              1
                                   ..
2094008980554    11. KLCC           1
                 12. Puchong        1
                 5. OKR             1
2094018631743    4. USJ             1
2094021746014    11. KLCC           1
Name: the_transaction_id, Length: 131145, dtype: int64


In [211]:
# Group the data by store and customer ID, and sum the sales generated
store_customer_sales = Temp_Data.groupby(['ctm_customer_id','store'])['f_to_tax_in'].sum()
store_customer_sales.head(10)

ctm_customer_id  store          
2090550311754    1. Decathlon.my    107.00
2090577786009    3. BSD             147.00
2090629397726    4. USJ             104.00
2090629795355    4. USJ             129.00
2090630810429    7. PJ               88.00
2090638740995    7. PJ              109.00
2090638798323    1. Decathlon.my     78.00
2090638872283    7. PJ             1654.00
2090639011896    4. USJ             198.00
2090639204779    11. KLCC            39.00
Name: f_to_tax_in, dtype: float64

In [212]:
print(store_customer_sales)

ctm_customer_id  store          
2090550311754    1. Decathlon.my   107.00
2090577786009    3. BSD            147.00
2090629397726    4. USJ            104.00
2090629795355    4. USJ            129.00
2090630810429    7. PJ              88.00
                                    ...  
2094008980554    11. KLCC            9.00
                 12. Puchong       123.00
                 5. OKR             69.00
2094018631743    4. USJ            321.00
2094021746014    11. KLCC          293.00
Name: f_to_tax_in, Length: 131145, dtype: float64


In [213]:
# Combine the two dataframes into one

store_customer_transactions = pd.DataFrame(store_customer_transactions)
store_customer_sales = pd.DataFrame(store_customer_sales)
store_customer_data = pd.concat([store_customer_transactions, store_customer_sales], axis=1)
store_customer_data.head()

,,the_transaction_id,f_to_tax_in
ctm_customer_id,store,,
2090550311754,1. Decathlon.my,1,107.00
2090577786009,3. BSD,1,147.00
2090629397726,4. USJ,1,104.00
2090629795355,4. USJ,1,129.00
2090630810429,7. PJ,1,88.00


In [214]:
store_customer_data

the_transaction_id  f_to_tax_in
ctm_customer_id store                                           
2090550311754   1. Decathlon.my                   1       107.00
2090577786009   3. BSD                            1       147.00
2090629397726   4. USJ                            1       104.00
2090629795355   4. USJ                            1       129.00
2090630810429   7. PJ                             1        88.00
...                                             ...          ...
2094008980554   11. KLCC                          1         9.00
                12. Puchong                       1       123.00
                5. OKR                            1        69.00
2094018631743   4. USJ                            1       321.00
2094021746014   11. KLCC                          1       293.00

[131145 rows x 2 columns]

### Pivoting : Transposing store to columns with respective no. of transaction & turnover

In [215]:
# Pivot the data to transpose the store column
store_customer_pivot_Transaction = store_customer_data.pivot_table(index='ctm_customer_id', columns='store', values=['the_transaction_id'])
store_customer_pivot_Turnover = store_customer_data.pivot_table(index='ctm_customer_id', columns='store', values=['f_to_tax_in'])


In [216]:

store_customer_pivot = store_customer_data.pivot_table(index='ctm_customer_id', columns='store', values=['the_transaction_id', 'f_to_tax_in'])

# Flatten the column names
store_customer_pivot.columns = ['_'.join(col).strip() for col in store_customer_pivot.columns.values]

store_customer_pivot.head()

,f_to_tax_in_1. Decathlon.my,f_to_tax_in_10. MyTown,f_to_tax_in_11. KLCC,f_to_tax_in_12. Puchong,f_to_tax_in_13. Cyberjaya,f_to_tax_in_3. BSD,f_to_tax_in_4. USJ,f_to_tax_in_5. OKR,f_to_tax_in_6. SA,f_to_tax_in_7. PJ,...,the_transaction_id_11. KLCC,the_transaction_id_12. Puchong,the_transaction_id_13. Cyberjaya,the_transaction_id_3. BSD,the_transaction_id_4. USJ,the_transaction_id_5. OKR,the_transaction_id_6. SA,the_transaction_id_7. PJ,the_transaction_id_8. KL EAST,the_transaction_id_st
ctm_customer_id,,,,,,,,,,,,,,,,,,,,,
2090550311754,107.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2090577786009,NaN,NaN,NaN,NaN,NaN,147.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN
2090629397726,NaN,NaN,NaN,NaN,NaN,NaN,104.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN
2090629795355,NaN,NaN,NaN,NaN,NaN,NaN,129.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN
2090630810429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN


### Pivoting : Getting the base store based on the no. of transaction and also label base store based on max turnover for members with similar no. of transaction by store

In [217]:
# Find the store with the highest transaction for each customer

idx = store_customer_transactions.groupby(['ctm_customer_id'])['the_transaction_id'].transform(max) == store_customer_transactions['the_transaction_id']
Cust_with_HighestTransacted_Store=store_customer_transactions[idx]

In [218]:
Cust_with_HighestTransacted_Store=pd.DataFrame(Cust_with_HighestTransacted_Store).reset_index()
len(Cust_with_HighestTransacted_Store)

130021

In [219]:
#Cust_with_HighestTransacted_Store.drop_duplicates('ctm_customer_id')
Cust_with_HighestTransacted_Store.drop_duplicates(subset='ctm_customer_id', keep='first', inplace=True)
#Cust_with_HighestTransacted_Store = Cust_with_HighestTransacted_Store.drop_duplicates()
#Cust_with_HighestTransacted_Store=pd.DataFrame(Cust_with_HighestTransacted_Store)

In [220]:
len(Cust_with_HighestTransacted_Store)

126304

In [221]:
# Find the store with the highest total sales generated among the stores with the highest transaction for each customer
store_customer_max_sales = Temp_Data.groupby(['ctm_customer_id', 'store'])['f_to_tax_in'].sum().reset_index(name='Max_sales')
#store_customer_max_sales = store_customer_max_sales.loc[store_customer_max_sales['store'].isin(store_customer_max_trans[idx]['store'])]
store_customer_max_sales_idx = store_customer_max_sales.groupby(['ctm_customer_id'])['Max_sales'].transform(max) == store_customer_max_sales['Max_sales']


store_customer_Highest_Sales = store_customer_max_sales[store_customer_max_sales_idx]
# Combine the results into a single dataframe
#store_customer_max = pd.merge(store_customer_max_trans[idx], store_customer_max_sales[store_customer_max_sales_idx], on=['ctm_customer_id', 'store'], how='inner')


In [222]:
store_customer_Highest_Sales

,ctm_customer_id,store,Max_sales
0,2090550311754,1. Decathlon.my,107.00
1,2090577786009,3. BSD,147.00
2,2090629397726,4. USJ,104.00
3,2090629795355,4. USJ,129.00
4,2090630810429,7. PJ,88.00
...,...,...,...
131137,2094002342525,4. USJ,830.00
131138,2094007252102,1. Decathlon.my,113.00
131141,2094008980554,12. Puchong,123.00
131143,2094018631743,4. USJ,321.00


In [223]:
store_customer_Highest_Sales.drop_duplicates(subset='ctm_customer_id', keep='first', inplace=True)
len(store_customer_Highest_Sales)

C:\Users\AAMIN01\AppData\Local\Temp\ipykernel_31996\1929176710.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  store_customer_Highest_Sales.drop_duplicates(subset='ctm_customer_id', keep='first', inplace=True)


126304

In [224]:
# Combine the results into a single dataframe
store_customer_max = pd.merge(Cust_with_HighestTransacted_Store,store_customer_Highest_Sales, on=['ctm_customer_id'], how='inner')
store_customer_max 

,ctm_customer_id,store_x,the_transaction_id,store_y,Max_sales
0,2090550311754,1. Decathlon.my,1,1. Decathlon.my,107.00
1,2090577786009,3. BSD,1,3. BSD,147.00
2,2090629397726,4. USJ,1,4. USJ,104.00
3,2090629795355,4. USJ,1,4. USJ,129.00
4,2090630810429,7. PJ,1,7. PJ,88.00
...,...,...,...,...,...
126299,2094002342525,4. USJ,2,4. USJ,830.00
126300,2094007252102,1. Decathlon.my,1,1. Decathlon.my,113.00
126301,2094008980554,10. MyTown,1,12. Puchong,123.00
126302,2094018631743,4. USJ,1,4. USJ,321.00


In [225]:
store_customer_max.rename(columns = {'store_x':'HTrans_Store', 'store_y':'HSales_Store'}, inplace = True)
store_customer_max


,ctm_customer_id,HTrans_Store,the_transaction_id,HSales_Store,Max_sales
0,2090550311754,1. Decathlon.my,1,1. Decathlon.my,107.00
1,2090577786009,3. BSD,1,3. BSD,147.00
2,2090629397726,4. USJ,1,4. USJ,104.00
3,2090629795355,4. USJ,1,4. USJ,129.00
4,2090630810429,7. PJ,1,7. PJ,88.00
...,...,...,...,...,...
126299,2094002342525,4. USJ,2,4. USJ,830.00
126300,2094007252102,1. Decathlon.my,1,1. Decathlon.my,113.00
126301,2094008980554,10. MyTown,1,12. Puchong,123.00
126302,2094018631743,4. USJ,1,4. USJ,321.00


In [226]:
# Combine the results into a single dataframe
Final = pd.merge(store_customer_max,store_customer_pivot, on=['ctm_customer_id'], how='inner')

In [227]:
Final

,ctm_customer_id,HTrans_Store,the_transaction_id,HSales_Store,Max_sales,f_to_tax_in_1. Decathlon.my,f_to_tax_in_10. MyTown,f_to_tax_in_11. KLCC,f_to_tax_in_12. Puchong,f_to_tax_in_13. Cyberjaya,...,the_transaction_id_11. KLCC,the_transaction_id_12. Puchong,the_transaction_id_13. Cyberjaya,the_transaction_id_3. BSD,the_transaction_id_4. USJ,the_transaction_id_5. OKR,the_transaction_id_6. SA,the_transaction_id_7. PJ,the_transaction_id_8. KL EAST,the_transaction_id_st
0,2090550311754,1. Decathlon.my,1,1. Decathlon.my,107.00,107.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2090577786009,3. BSD,1,3. BSD,147.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN
2,2090629397726,4. USJ,1,4. USJ,104.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN
3,2090629795355,4. USJ,1,4. USJ,129.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN
4,2090630810429,7. PJ,1,7. PJ,88.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126299,2094002342525,4. USJ,2,4. USJ,830.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.00,NaN,NaN,NaN,NaN,NaN
126300,2094007252102,1. Decathlon.my,1,1. Decathlon.my,113.00,113.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126301,2094008980554,10. MyTown,1,12. Puchong,123.00,NaN,35.00,9.00,123.00,NaN,...,1.00,1.00,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN
126302,2094018631743,4. USJ,1,4. USJ,321.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN


In [228]:
print(Final.dtypes)

ctm_customer_id                        object
HTrans_Store                           object
the_transaction_id                      int64
HSales_Store                           object
Max_sales                             float64
f_to_tax_in_1. Decathlon.my           float64
f_to_tax_in_10. MyTown                float64
f_to_tax_in_11. KLCC                  float64
f_to_tax_in_12. Puchong               float64
f_to_tax_in_13. Cyberjaya             float64
f_to_tax_in_3. BSD                    float64
f_to_tax_in_4. USJ                    float64
f_to_tax_in_5. OKR                    float64
f_to_tax_in_6. SA                     float64
f_to_tax_in_7. PJ                     float64
f_to_tax_in_8. KL EAST                float64
f_to_tax_in_st                        float64
the_transaction_id_1. Decathlon.my    float64
the_transaction_id_10. MyTown         float64
the_transaction_id_11. KLCC           float64
the_transaction_id_12. Puchong        float64
the_transaction_id_13. Cyberjaya  

If the values is 1 , then the max_trans column have the highest transaction above others , if there is other value similar to highest trans value , it will return 0 

In [229]:
Final['Result'] = (Final.iloc[:, 18:30].values < Final[['the_transaction_id']].values).all(axis=1).astype(int)
print (Final)

       ctm_customer_id     HTrans_Store  the_transaction_id     HSales_Store  \
0        2090550311754  1. Decathlon.my                   1  1. Decathlon.my   
1        2090577786009           3. BSD                   1           3. BSD   
2        2090629397726           4. USJ                   1           4. USJ   
3        2090629795355           4. USJ                   1           4. USJ   
4        2090630810429            7. PJ                   1            7. PJ   
...                ...              ...                 ...              ...   
126299   2094002342525           4. USJ                   2           4. USJ   
126300   2094007252102  1. Decathlon.my                   1  1. Decathlon.my   
126301   2094008980554       10. MyTown                   1      12. Puchong   
126302   2094018631743           4. USJ                   1           4. USJ   
126303   2094021746014         11. KLCC                   1         11. KLCC   

        Max_sales  f_to_tax_in_1. Decat

In [230]:
Final['BaseStore'] = Final.apply(lambda x: x['HSales_Store'] if x['Result'] == 0 else x['HTrans_Store'], axis=1)

In [231]:
Final

,ctm_customer_id,HTrans_Store,the_transaction_id,HSales_Store,Max_sales,f_to_tax_in_1. Decathlon.my,f_to_tax_in_10. MyTown,f_to_tax_in_11. KLCC,f_to_tax_in_12. Puchong,f_to_tax_in_13. Cyberjaya,...,the_transaction_id_13. Cyberjaya,the_transaction_id_3. BSD,the_transaction_id_4. USJ,the_transaction_id_5. OKR,the_transaction_id_6. SA,the_transaction_id_7. PJ,the_transaction_id_8. KL EAST,the_transaction_id_st,Result,BaseStore
0,2090550311754,1. Decathlon.my,1,1. Decathlon.my,107.00,107.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1. Decathlon.my
1,2090577786009,3. BSD,1,3. BSD,147.00,NaN,NaN,NaN,NaN,NaN,...,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,0,3. BSD
2,2090629397726,4. USJ,1,4. USJ,104.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,0,4. USJ
3,2090629795355,4. USJ,1,4. USJ,129.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,0,4. USJ
4,2090630810429,7. PJ,1,7. PJ,88.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,0,7. PJ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126299,2094002342525,4. USJ,2,4. USJ,830.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.00,NaN,NaN,NaN,NaN,NaN,0,4. USJ
126300,2094007252102,1. Decathlon.my,1,1. Decathlon.my,113.00,113.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1. Decathlon.my
126301,2094008980554,10. MyTown,1,12. Puchong,123.00,NaN,35.00,9.00,123.00,NaN,...,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,0,12. Puchong
126302,2094018631743,4. USJ,1,4. USJ,321.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,0,4. USJ


In [232]:
len(Final)

126304

In [233]:
FilteredFinal = Final.loc[:,['ctm_customer_id','HTrans_Store']]
len(FilteredFinal)

126304

In [234]:
FilteredFinal.to_csv(str(dir_)+'/data/2022_11members.csv', index=False)
Final.to_csv(str(dir_)+'/data/alldata_2022_11members.csv', index=False)

In [235]:
#df['Result'] = (df.iloc[:, 3:4].values < df[['Column2']].values).all(axis=1).astype(int)
#print (df)

In [236]:
#saving files by turnover



In [237]:
FilteredFinal = Final.loc[:,['ctm_customer_id','HSales_Store','Max_sales']]
len(FilteredFinal)

126304

In [238]:
FilteredFinal.to_csv(str(dir_)+'/data/2022_11byturnover.csv', index=False)
Final.to_csv(str(dir_)+'/data/alldata_2022_11byturnover.csv', index=False)